In [1]:
!head -3 /media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_user_all/20131228/part-00000

['2013-12-28 16:28:25', '8613032261669', '120.198.203.152', 'web', 'dns.weixin.qq.com', 'qq', 'MicroMessenger Client', '4366', '6', '5', '7', '1827', 'ƻ��']
['2013-12-28 16:28:35', '8613032261669', '111.30.135.151', 'web', 'dns.weixin.qq.com', 'qq', 'MicroMessenger Client', '9920', '20', '13', '7', '1827', 'ƻ��']
['2013-12-28 16:28:34', '8613032261669', '117.135.169.19', 'web', 'short.weixin.qq.com', 'qq', 'Mozilla/4.0', '12392', '8', '6', '7', '1827', 'ƻ��']


# 筛选出四天都有数据的一部分用户来做进一步分析

In [6]:
!head -5 /media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_user_times/20131229/part-00003

('8615895961950', 149)
('8615950593628', 149)
('8613824297918', 149)
('8615850506821', 149)
('8613814145811', 149)


In [50]:
#qq_user_times_filepath = "t*.txt"
qq_user_times_filepath = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_user_times/*/part-*"

In [51]:
qq_user_times = sc.textFile(qq_user_times_filepath) # 载入所有QQ用户12.28-12.31四天访问qq的次数信息

In [52]:
# 将每个qq用户4天的访问次数和到一个list里面
qq_user_times_4days = qq_user_times.map(lambda line: eval(line)).groupByKey().mapValues(list) 
# qq_user_times_4days: ('8615905164699', [64, 1, 42]) ('8613585154570', [182, 11, 25])

In [54]:
qq_user_times_4days.take(10)

[('8615905164699', [64, 1, 42]),
 ('8613585154570', [182, 11, 25]),
 ('8613605164420', [20, 24, 25, 6]),
 ('8613770626973', [26, 11, 93, 143]),
 ('8613601463848', [5]),
 ('8615996410957', [16, 63, 181, 28]),
 ('8618738607127', [3]),
 ('8615751886018', [127, 736, 118, 56]),
 ('8613705169075', [7]),
 ('8615751886086', [52, 104, 48, 113])]

In [55]:
qq_user_4days_yes = qq_user_times_4days.filter(lambda x: len(x[1]) == 4) # 过滤出四天都有数据的QQ用户及访问次数
# qq_user_4days_yes: ('8613605164420', [20, 24, 25, 6]),('8613770626973', [26, 11, 93, 143])

In [57]:
qq_user_4days_yes.take(5)

[('8613605164420', [20, 24, 25, 6]),
 ('8613770626973', [26, 11, 93, 143]),
 ('8615996410957', [16, 63, 181, 28]),
 ('8615751886018', [127, 736, 118, 56]),
 ('8615751886086', [52, 104, 48, 113])]

In [58]:
qq_user_4days_yes.count()  # 四天都有数据的用户共有1384149

1384149

In [59]:
# 4天都有数据的用户存到文件,格式：('8613584029812', [236, 83, 183, 269])
qq_user_4days_yes.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_user_4days_times")

In [64]:
# 取5000个用户做进一步分析,####未排序，随机选的####
qq_user_4days_yes_5000 = qq_user_4days_yes.take(5000)

In [65]:
qq_user_4days_yes_5000[-1]

('8613584029812', [236, 83, 183, 269])

In [66]:
# 将这5000个用户保存到文件。格式：('8613584029812', [236, 83, 183, 269])
sc.parallelize(qq_user_4days_yes_5000).saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_user_times_5000")

In [71]:
# 将这5000个用户的号码保存到文件。格式：'8613605164420'
qq_user_5000 = sc.parallelize(qq_user_4days_yes_5000).map(lambda x: x[0])
qq_user_5000.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_user_phone_5000")

In [72]:
qq_user_5000.take(5)

['8613605164420',
 '8613770626973',
 '8615996410957',
 '8615751886018',
 '8615751886086']

In [75]:
qq_user_5000_list = qq_user_5000.collect()
print(qq_user_5000_list[-1])

8613584029812


### 筛选出这5000个用户的dpi记录

In [20]:
# 5000个QQ用户的id
qq_user_5000_list = sc.textFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_user_phone_5000").collect()

In [31]:
qq_user_all_filepath = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_user_all/20131231/part-*"

In [32]:
qq_user_all = sc.textFile(qq_user_all_filepath)

In [33]:
# 过滤出这5000个用户的dpi记录并保存到文件
qq_user_5000_dpi = qq_user_all.map(lambda x: eval(x)).filter(lambda x: x[1] in qq_user_5000_list)

In [34]:
'''print(qq_user_5000_dpi.count())
print(qq_user_5000_dpi.take(3))'''

'print(qq_user_5000_dpi.count())\nprint(qq_user_5000_dpi.take(3))'

In [35]:
qq_user_5000_dpi.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_dpi/20131231")